In [89]:
import pandas as pd
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import openai
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [90]:
df = pd.read_csv("/Users/shubh/Downloads/Team05.csv")

In [91]:
openai.api_key = os.getenv('Openaikey')

In [330]:
def generate_qa(summary, learningoutcomes):
    set_a = ""
    set_b = ""

    for i in range(5):
        prompt = f"""Create 50 multiple-choice questions for a financial analyst with an MBA from summary section based on learning outcomes: {learningoutcomes[0]}. Include 4 options per question with one correct answer and immediate answers, maintaining uniform complexity. Make sure to include tables, formulas and relevant information if necessary. Start the numberings from {i*30+1} to {(i+1)*30}.
        Keep consistent with the following format:
        1. Question
        A. Option 1
        B. Option 2
        C. Option 3
        D. Option 4
        Answer: A (Correct Answer)
        """

        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=3649,
            n=1,
            stop=None,
            temperature=0.49,
        )

        set_a += response.choices[0].text.strip() + "\n\n"

        prompt = f"""Create 50 multiple-choice questions for a financial analyst with an MBA from summary section based on learning outcomes: {learningoutcomes}. Include 4 options per question with one correct answer and immediate answers, maintaining uniform complexity. Make sure to include tables, formulas and relevant information if necessary. The questions should be elaborate. Start the numberings from {i*30+1} to {(i+1)*30}.
        Keep consistent with the following format:
        1. Question
        A. Option 1
        B. Option 2
        C. Option 3
        D. Option 4
        Answer: A (Correct Answer)
        """

        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=3649,
            n=1,
            stop=None,
            temperature=0.49,
        )

        set_b += response.choices[0].text.strip() + "\n\n"

    return set_a.strip(), set_b.strip()

In [331]:
OPENAI_API_KEY = os.getenv('Openaikey')

In [332]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [333]:
summary = df["Summary"][0]
learningoutcomes = df["LearningOutcomes"][0]

set_a, set_b = generate_qa(summary, learningoutcomes)

In [336]:
# write to file
with open("set_a.txt", "w") as f:
    f.write(set_a)
    
with open("set_b.txt", "w") as f:
    f.write(set_b)

In [335]:
set_a

"1. What is the primary goal of financial analysis?\nA. Maximizing profits\nB. Minimizing expenses\nC. Maximizing shareholder wealth\nD. Minimizing risk\nAnswer: C (Maximizing shareholder wealth)\n\n2. Which of the following statements is true about financial statements?\nA. They provide information about a company's past performance.\nB. They predict a company's future performance.\nC. They are only used by external stakeholders.\nD. They are not affected by economic conditions.\nAnswer: A (They provide information about a company's past performance)\n\n3. Which financial statement reports a company's revenues and expenses over a period of time?\nA. Balance sheet\nB. Income statement\nC. Cash flow statement\nD. Statement of retained earnings\nAnswer: B (Income statement)\n\n4. What is the formula for calculating return on equity (ROE)?\nA. Net income / total assets\nB. Net income / total equity\nC. Total assets / total equity\nD. Total equity / net income\nAnswer: B (Net income / tota

In [337]:
import re

set_a_lines = set_a.split("\n\n")
set_a_questions = [line.split("\n")[0] for line in set_a_lines]

In [338]:
set_a_questions

['1. What is the primary goal of financial analysis?',
 '2. Which of the following statements is true about financial statements?',
 "3. Which financial statement reports a company's revenues and expenses over a period of time?",
 '4. What is the formula for calculating return on equity (ROE)?',
 "5. Which financial ratio measures a company's ability to pay its short-term obligations?",
 '6. What is the formula for calculating earnings per share (EPS)?',
 '7. Which of the following is NOT a component of the DuPont analysis?',
 '8. How is the cost of goods sold (COGS) calculated?',
 '9. What is the formula for calculating the debt-to-equity ratio?',
 '10. Which of the following is NOT a type of financial statement analysis?',
 '11. What is the purpose of conducting a horizontal analysis?',
 '12. Which of the following is NOT a limitation of financial statement analysis?',
 '13. What is the formula for calculating the present value of a future cash flow?',
 '14. Which of the following is

In [339]:
len(set_a_questions)

150

In [340]:
# save set_a_questions to a file
with open("set_a_questions.txt", "w") as f:
    for question in set_a_questions:
        f.write(question + "\n")

In [341]:
set_a_lines

['1. What is the primary goal of financial analysis?\nA. Maximizing profits\nB. Minimizing expenses\nC. Maximizing shareholder wealth\nD. Minimizing risk\nAnswer: C (Maximizing shareholder wealth)',
 "2. Which of the following statements is true about financial statements?\nA. They provide information about a company's past performance.\nB. They predict a company's future performance.\nC. They are only used by external stakeholders.\nD. They are not affected by economic conditions.\nAnswer: A (They provide information about a company's past performance)",
 "3. Which financial statement reports a company's revenues and expenses over a period of time?\nA. Balance sheet\nB. Income statement\nC. Cash flow statement\nD. Statement of retained earnings\nAnswer: B (Income statement)",
 '4. What is the formula for calculating return on equity (ROE)?\nA. Net income / total assets\nB. Net income / total equity\nC. Total assets / total equity\nD. Total equity / net income\nAnswer: B (Net income / 

In [342]:
set_a_questions

['1. What is the primary goal of financial analysis?',
 '2. Which of the following statements is true about financial statements?',
 "3. Which financial statement reports a company's revenues and expenses over a period of time?",
 '4. What is the formula for calculating return on equity (ROE)?',
 "5. Which financial ratio measures a company's ability to pay its short-term obligations?",
 '6. What is the formula for calculating earnings per share (EPS)?',
 '7. Which of the following is NOT a component of the DuPont analysis?',
 '8. How is the cost of goods sold (COGS) calculated?',
 '9. What is the formula for calculating the debt-to-equity ratio?',
 '10. Which of the following is NOT a type of financial statement analysis?',
 '11. What is the purpose of conducting a horizontal analysis?',
 '12. Which of the following is NOT a limitation of financial statement analysis?',
 '13. What is the formula for calculating the present value of a future cash flow?',
 '14. Which of the following is

In [343]:
set_a_only_questions = set_a_questions

In [344]:
# extract the answers with the correct answer in text from set_a_lines
set_a_answers = [line for line in set_a_lines if "Answer:" in line]
set_a_answers = [re.split(r"Answer:|\n", line) for line in set_a_answers]
set_a_answers = [line[1].strip() for line in set_a_answers]

In [345]:
set_a_answers

['A. Maximizing profits',
 "A. They provide information about a company's past performance.",
 'A. Balance sheet',
 'A. Net income / total assets',
 'A. Debt-to-equity ratio',
 'A. Net income / total assets',
 'A. Return on assets',
 'A. Beginning inventory + purchases - ending inventory',
 'A. Total liabilities / total equity',
 'A. Vertical analysis',
 "A. To compare a company's financial performance to industry averages",
 'A. It relies on historical data.',
 'A. Future value / (1 + r)^n',
 'A. Payback period',
 'A. Initial investment / annual cash flow',
 'A. Risk-free rate',
 'A. Risk-free rate + beta coefficient * market risk premium',
 'A. Market risk',
 'A. (Cost of equity * weight of equity) + (cost of debt * weight of debt)',
 'A. Dividend growth rate',
 'A. Return on equity * retention ratio',
 'A. Primary market',
 'A. Market price per share / earnings per share',
 'A. Dow Jones Industrial Average (DJIA)',
 'A. Annual dividends per share / market price per share',
 'A. Comm

In [346]:
len(set_a_answers)

150

In [347]:
from langchain_pinecone import PineconeVectorStore

# index_name = "assignment05"

In [348]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key= os.getenv('PINECONE_API_KEY'),
    environment="us-east1-gcp"
)



In [350]:
set_a_only_questions

['1. What is the primary goal of financial analysis?',
 '2. Which of the following statements is true about financial statements?',
 "3. Which financial statement reports a company's revenues and expenses over a period of time?",
 '4. What is the formula for calculating return on equity (ROE)?',
 "5. Which financial ratio measures a company's ability to pay its short-term obligations?",
 '6. What is the formula for calculating earnings per share (EPS)?',
 '7. Which of the following is NOT a component of the DuPont analysis?',
 '8. How is the cost of goods sold (COGS) calculated?',
 '9. What is the formula for calculating the debt-to-equity ratio?',
 '10. Which of the following is NOT a type of financial statement analysis?',
 '11. What is the purpose of conducting a horizontal analysis?',
 '12. Which of the following is NOT a limitation of financial statement analysis?',
 '13. What is the formula for calculating the present value of a future cash flow?',
 '14. Which of the following is

In [351]:
set_a_answers

['A. Maximizing profits',
 "A. They provide information about a company's past performance.",
 'A. Balance sheet',
 'A. Net income / total assets',
 'A. Debt-to-equity ratio',
 'A. Net income / total assets',
 'A. Return on assets',
 'A. Beginning inventory + purchases - ending inventory',
 'A. Total liabilities / total equity',
 'A. Vertical analysis',
 "A. To compare a company's financial performance to industry averages",
 'A. It relies on historical data.',
 'A. Future value / (1 + r)^n',
 'A. Payback period',
 'A. Initial investment / annual cash flow',
 'A. Risk-free rate',
 'A. Risk-free rate + beta coefficient * market risk premium',
 'A. Market risk',
 'A. (Cost of equity * weight of equity) + (cost of debt * weight of debt)',
 'A. Dividend growth rate',
 'A. Return on equity * retention ratio',
 'A. Primary market',
 'A. Market price per share / earnings per share',
 'A. Dow Jones Industrial Average (DJIA)',
 'A. Annual dividends per share / market price per share',
 'A. Comm

In [352]:
index_name = "assignment05"

# Create the vector store for the questions
docsearch = PineconeVectorStore.from_texts(
    set_a_only_questions,
    embeddings,
    index_name=index_name,
    namespace="questions",
    metadatas=[{"id": i} for i in range(len(set_a_only_questions))]
)


In [353]:
from pinecone import Pinecone

pc = Pinecone(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment="us-east1-gcp"
)
index = pc.Index(index_name)

In [354]:
all_ids = []
for ids in index.list(namespace='questions'):
    for id in ids:
        all_ids.append(id)

In [355]:
set_a_only_questions

['1. What is the primary goal of financial analysis?',
 '2. Which of the following statements is true about financial statements?',
 "3. Which financial statement reports a company's revenues and expenses over a period of time?",
 '4. What is the formula for calculating return on equity (ROE)?',
 "5. Which financial ratio measures a company's ability to pay its short-term obligations?",
 '6. What is the formula for calculating earnings per share (EPS)?',
 '7. Which of the following is NOT a component of the DuPont analysis?',
 '8. How is the cost of goods sold (COGS) calculated?',
 '9. What is the formula for calculating the debt-to-equity ratio?',
 '10. Which of the following is NOT a type of financial statement analysis?',
 '11. What is the purpose of conducting a horizontal analysis?',
 '12. Which of the following is NOT a limitation of financial statement analysis?',
 '13. What is the formula for calculating the present value of a future cash flow?',
 '14. Which of the following is

In [356]:
all_ids

['042f4ca2-6e1e-46b4-b0d2-f1ede3c538a5',
 '04e0e172-ac0b-4e8c-a378-f9d6286bac00',
 '059f3ece-f8d4-4bea-a0d5-001e04b01382',
 '06130eb5-401b-4752-9e7f-f49099ef8dec',
 '0657d247-f7f1-48af-a5c5-7856a1d68164',
 '070171dc-ae74-4be3-ba3b-2a075fe16c2a',
 '08275f80-6c60-4b65-9ee0-b742ec84acfe',
 '0b3ba815-ecba-4882-b389-901519d2b098',
 '0c8c5418-659d-4d7a-b520-2e120247483d',
 '0ddb8045-ca2c-44f9-aa05-16307fe4c814',
 '0e2ecba2-9612-42c4-8d0c-4458cbe6f248',
 '0f1fc5ee-98d4-4ada-9c23-5ff9c0931180',
 '0f87a68a-1dfb-4018-9937-f50c76bcdcf2',
 '114dd48c-7fd8-4fb3-adb6-bbc637607b46',
 '148bb7c7-0e01-456a-8050-6d52acc185bd',
 '14916f2b-ccc0-4d75-b531-082db15920e0',
 '1545d5aa-046a-4abc-b0f1-4dea9432d693',
 '1b150815-be83-4497-a492-7fb691aa7736',
 '1b331961-7915-4ef3-89fc-06719e44812b',
 '1d40e291-4126-42b3-8613-82d3458e27f9',
 '1de9836e-2115-43a3-8771-15797166ceaf',
 '1fc3ef5a-f26a-4945-9641-9134f4e2a1a3',
 '21fbdcfd-a882-4507-a44c-43cd72438f85',
 '228a47bb-b2cb-451b-b69e-ab0af774f3ae',
 '23e9b580-7214-

In [357]:
all_questions = []

for i in all_ids:
    all_questions.append(index.query(id=i, top_k=1, namespace='questions', include_metadata=True))

In [358]:
all_questions

[{'matches': [{'id': '042f4ca2-6e1e-46b4-b0d2-f1ede3c538a5',
               'metadata': {'id': 112.0,
                            'text': "113. A company's asset turnover ratio is "
                                    'calculated by dividing its net sales by '
                                    "its total assets. If a company's net sales "
                                    'are $1,000,000 and its total assets are '
                                    '$500,000, what is its asset turnover '
                                    'ratio?'},
               'score': 1.00000012,
               'values': []}],
  'namespace': 'questions',
  'usage': {'read_units': 6}},
 {'matches': [{'id': '04e0e172-ac0b-4e8c-a378-f9d6286bac00',
               'metadata': {'id': 100.0,
                            'text': "101. A company's return on assets (ROA) is "
                                    'calculated by dividing its net income by '
                                    "its total assets. If a compa

In [359]:
all_questions[0].matches[0].id

'042f4ca2-6e1e-46b4-b0d2-f1ede3c538a5'

In [360]:
all_questions[0].matches[0].metadata['id']

112.0

In [361]:
# map the question ids to the questions id
question_id_to_question = {}
for question in all_questions:
    question_id_to_question[question.matches[0].metadata['id']] = question.matches[0].id

In [362]:
question_id_to_question

{112.0: '042f4ca2-6e1e-46b4-b0d2-f1ede3c538a5',
 100.0: '04e0e172-ac0b-4e8c-a378-f9d6286bac00',
 11.0: '059f3ece-f8d4-4bea-a0d5-001e04b01382',
 1.0: '06130eb5-401b-4752-9e7f-f49099ef8dec',
 92.0: '0657d247-f7f1-48af-a5c5-7856a1d68164',
 133.0: '070171dc-ae74-4be3-ba3b-2a075fe16c2a',
 57.0: '08275f80-6c60-4b65-9ee0-b742ec84acfe',
 130.0: '0b3ba815-ecba-4882-b389-901519d2b098',
 111.0: '0c8c5418-659d-4d7a-b520-2e120247483d',
 135.0: '0ddb8045-ca2c-44f9-aa05-16307fe4c814',
 122.0: '0e2ecba2-9612-42c4-8d0c-4458cbe6f248',
 120.0: '0f1fc5ee-98d4-4ada-9c23-5ff9c0931180',
 127.0: '0f87a68a-1dfb-4018-9937-f50c76bcdcf2',
 31.0: '114dd48c-7fd8-4fb3-adb6-bbc637607b46',
 59.0: '148bb7c7-0e01-456a-8050-6d52acc185bd',
 7.0: '14916f2b-ccc0-4d75-b531-082db15920e0',
 28.0: '1545d5aa-046a-4abc-b0f1-4dea9432d693',
 36.0: '1b150815-be83-4497-a492-7fb691aa7736',
 131.0: '1b331961-7915-4ef3-89fc-06719e44812b',
 61.0: '1d40e291-4126-42b3-8613-82d3458e27f9',
 47.0: '1de9836e-2115-43a3-8771-15797166ceaf',
 113.

In [363]:
question_id_to_question.values(),

(dict_values(['042f4ca2-6e1e-46b4-b0d2-f1ede3c538a5', '04e0e172-ac0b-4e8c-a378-f9d6286bac00', '059f3ece-f8d4-4bea-a0d5-001e04b01382', '06130eb5-401b-4752-9e7f-f49099ef8dec', '0657d247-f7f1-48af-a5c5-7856a1d68164', '070171dc-ae74-4be3-ba3b-2a075fe16c2a', '08275f80-6c60-4b65-9ee0-b742ec84acfe', '0b3ba815-ecba-4882-b389-901519d2b098', '0c8c5418-659d-4d7a-b520-2e120247483d', '0ddb8045-ca2c-44f9-aa05-16307fe4c814', '0e2ecba2-9612-42c4-8d0c-4458cbe6f248', '0f1fc5ee-98d4-4ada-9c23-5ff9c0931180', '0f87a68a-1dfb-4018-9937-f50c76bcdcf2', '114dd48c-7fd8-4fb3-adb6-bbc637607b46', '148bb7c7-0e01-456a-8050-6d52acc185bd', '14916f2b-ccc0-4d75-b531-082db15920e0', '1545d5aa-046a-4abc-b0f1-4dea9432d693', '1b150815-be83-4497-a492-7fb691aa7736', '1b331961-7915-4ef3-89fc-06719e44812b', '1d40e291-4126-42b3-8613-82d3458e27f9', '1de9836e-2115-43a3-8771-15797166ceaf', '1fc3ef5a-f26a-4945-9641-9134f4e2a1a3', '21fbdcfd-a882-4507-a44c-43cd72438f85', '228a47bb-b2cb-451b-b69e-ab0af774f3ae', '23e9b580-7214-4ac7-8f70-3

In [364]:
# Create the vector store for the answers
docsearch_answers = PineconeVectorStore.from_texts(
    set_a_answers,
    embeddings,
    ids=[question_id_to_question[i] for i in range(len(set_a_answers))],
    index_name=index_name,
    namespace="answers",
    metadatas=[{"question_id": question_id_to_question[i]} for i in range(len(set_a_answers))]
)

In [365]:
set_b_lines = set_b.split("\n\n")
set_b_questions = [line.split("\n")[0] for line in set_b_lines]

In [366]:
set_b_lines

['1. What is the primary role of equities in an overall portfolio?\nA. To provide stable income\nB. To reduce risk\nC. To generate capital gains\nD. To diversify the portfolio\nAnswer: C (To generate capital gains)',
 "2. Which of the following is NOT a way to segment an equity manager's investment universe?\nA. Industry\nB. Market capitalization\nC. Geographical location\nD. Credit rating\nAnswer: D (Credit rating)",
 '3. What are the potential effects of owning and managing an equity portfolio?\nA. Increased liquidity risk\nB. Lower transaction costs\nC. Higher tax liabilities\nD. Decreased portfolio volatility\nAnswer: C (Higher tax liabilities)',
 '4. Which of the following is NOT a type of income associated with owning an equity portfolio?\nA. Dividends\nB. Interest payments\nC. Capital gains\nD. Coupon payments\nAnswer: D (Coupon payments)',
 '5. What is the potential effect of high portfolio turnover on portfolio performance?\nA. Increased transaction costs\nB. Lower tax liabili

In [367]:
set_b_lines[0].split("\n")

['1. What is the primary role of equities in an overall portfolio?',
 'A. To provide stable income',
 'B. To reduce risk',
 'C. To generate capital gains',
 'D. To diversify the portfolio',
 'Answer: C (To generate capital gains)']

In [368]:
# create b options list
set_b_options = [line.split("\n")[1:5] for line in set_b_lines]

In [369]:
set_b_questions

['1. What is the primary role of equities in an overall portfolio?',
 "2. Which of the following is NOT a way to segment an equity manager's investment universe?",
 '3. What are the potential effects of owning and managing an equity portfolio?',
 '4. Which of the following is NOT a type of income associated with owning an equity portfolio?',
 '5. What is the potential effect of high portfolio turnover on portfolio performance?',
 '6. How can an equity manager engage with shareholders?',
 '7. What is the primary benefit of shareholder engagement?',
 '8. What is the role of an equity manager in shareholder engagement?',
 '9. Which of the following is NOT a rationale for passive equity investment?',
 '10. What is the primary rationale for active equity investment?',
 '11. Which of the following is NOT a benefit of passive equity investment?',
 '12. How is portfolio turnover calculated?',
 '13. What is the primary benefit of lower portfolio turnover?',
 '14. What is the main difference bet

In [370]:
set_b_options

[['A. To provide stable income',
  'B. To reduce risk',
  'C. To generate capital gains',
  'D. To diversify the portfolio'],
 ['A. Industry',
  'B. Market capitalization',
  'C. Geographical location',
  'D. Credit rating'],
 ['A. Increased liquidity risk',
  'B. Lower transaction costs',
  'C. Higher tax liabilities',
  'D. Decreased portfolio volatility'],
 ['A. Dividends',
  'B. Interest payments',
  'C. Capital gains',
  'D. Coupon payments'],
 ['A. Increased transaction costs',
  'B. Lower tax liabilities',
  'C. Higher portfolio volatility',
  'D. Decreased liquidity risk'],
 ['A. By attending shareholder meetings',
  'B. By voting on shareholder proposals',
  'C. By communicating with company management',
  'D. All of the above'],
 ['A. Increased portfolio diversification',
  'B. Higher portfolio returns',
  'C. Improved corporate governance',
  'D. Reduced portfolio risk'],
 ['A. To represent the interests of shareholders',
  'B. To negotiate with company management',
  'C. To

In [371]:
set_b_answer = [line.split("\n")[5] for line in set_b_lines]

In [372]:
set_b_questions

['1. What is the primary role of equities in an overall portfolio?',
 "2. Which of the following is NOT a way to segment an equity manager's investment universe?",
 '3. What are the potential effects of owning and managing an equity portfolio?',
 '4. Which of the following is NOT a type of income associated with owning an equity portfolio?',
 '5. What is the potential effect of high portfolio turnover on portfolio performance?',
 '6. How can an equity manager engage with shareholders?',
 '7. What is the primary benefit of shareholder engagement?',
 '8. What is the role of an equity manager in shareholder engagement?',
 '9. Which of the following is NOT a rationale for passive equity investment?',
 '10. What is the primary rationale for active equity investment?',
 '11. Which of the following is NOT a benefit of passive equity investment?',
 '12. How is portfolio turnover calculated?',
 '13. What is the primary benefit of lower portfolio turnover?',
 '14. What is the main difference bet

In [373]:
set_b_only_questions = set_b_questions

In [374]:
set_b_only_questions

['1. What is the primary role of equities in an overall portfolio?',
 "2. Which of the following is NOT a way to segment an equity manager's investment universe?",
 '3. What are the potential effects of owning and managing an equity portfolio?',
 '4. Which of the following is NOT a type of income associated with owning an equity portfolio?',
 '5. What is the potential effect of high portfolio turnover on portfolio performance?',
 '6. How can an equity manager engage with shareholders?',
 '7. What is the primary benefit of shareholder engagement?',
 '8. What is the role of an equity manager in shareholder engagement?',
 '9. Which of the following is NOT a rationale for passive equity investment?',
 '10. What is the primary rationale for active equity investment?',
 '11. Which of the following is NOT a benefit of passive equity investment?',
 '12. How is portfolio turnover calculated?',
 '13. What is the primary benefit of lower portfolio turnover?',
 '14. What is the main difference bet

In [375]:
set_b_only_questions_vectors = embeddings.embed_query(set_b_only_questions[0])


In [376]:
set_b_only_questions_vectors

[0.0022160536781708124,
 -0.016062321266116196,
 -0.012144364510923008,
 -0.03035440319510556,
 -0.00822640775572982,
 0.011994674894987591,
 -0.023234395490741303,
 0.010296026324727486,
 -0.004240114274770884,
 0.0018011537753240495,
 -0.01690839244146785,
 0.008395621618271117,
 0.002412927821393024,
 0.03306182872105665,
 -0.0025382111977485075,
 0.016114387786377044,
 0.01681727556535008,
 -0.019316438735548114,
 0.02798540539423706,
 -0.008284981426870045,
 -0.03842461139005284,
 -0.0018629819532265462,
 0.004972290751959194,
 -0.01754619939842128,
 -0.01306853360402077,
 0.007373828963837497,
 0.011389409280366546,
 -0.006602603527776132,
 0.00585090233832065,
 -0.011031456377355486,
 0.026514545288981062,
 0.016843309756803083,
 -0.004546001588843678,
 -0.00010382667177205402,
 -0.003553495769980169,
 0.02223212661725227,
 0.010855734432612228,
 0.017494132878160432,
 0.028922591583061322,
 -0.0028945370802090768,
 0.02668375915152236,
 0.018678631778594683,
 -0.014409230202592

In [377]:
question_t3 = index.query(vector=set_b_only_questions_vectors, top_k=3, namespace='questions', include_metadata=True, include_values=True)

In [378]:
question_t3

{'matches': [{'id': 'a9d61a7f-cbbd-4a30-951b-900b8fba04a9',
              'metadata': {'id': 0.0,
                           'text': '1. What is the primary goal of financial '
                                   'analysis?'},
              'score': 0.87138319,
              'values': [0.00107447733,
                         -0.00990854297,
                         0.00278796116,
                         -0.0149069922,
                         -0.0101231234,
                         -0.00444622198,
                         -0.034231808,
                         -0.00912595727,
                         -0.00297571858,
                         -0.0194636602,
                         -0.0116882939,
                         0.0363776088,
                         0.00707482593,
                         0.028627485,
                         -0.018655831,
                         0.00867786445,
                         0.0117640281,
                         -0.0146545451,
                     

In [379]:
answer_ids = []
for question in question_t3.matches: 
    answer_ids.append(index.query(vector=question.values, top_k=1, namespace='answers', include_metadata=True))

In [380]:
answer_ids

[{'matches': [{'id': 'c5dea197-a35d-47c1-8f34-bf41f8df37ea',
               'metadata': {'question_id': 'c5dea197-a35d-47c1-8f34-bf41f8df37ea',
                            'text': "A. To determine a company's profitability"},
               'score': 0.879140437,
               'values': []}],
  'namespace': 'answers',
  'usage': {'read_units': 6}},
 {'matches': [{'id': 'c5dea197-a35d-47c1-8f34-bf41f8df37ea',
               'metadata': {'question_id': 'c5dea197-a35d-47c1-8f34-bf41f8df37ea',
                            'text': "A. To determine a company's profitability"},
               'score': 0.876485884,
               'values': []}],
  'namespace': 'answers',
  'usage': {'read_units': 6}},
 {'matches': [{'id': '1de9836e-2115-43a3-8771-15797166ceaf',
               'metadata': {'question_id': '1de9836e-2115-43a3-8771-15797166ceaf',
                            'text': 'A. Debt-to-equity ratio'},
               'score': 0.921255589,
               'values': []}],
  'namespace': 'answer

In [381]:
answers_gpt = []
for answer_id in answer_ids:
    answers_gpt.append(answer_id.matches[0].metadata['text'])

In [382]:
answers_gpt

["A. To determine a company's profitability",
 "A. To determine a company's profitability",
 'A. Debt-to-equity ratio']

In [401]:
def ask_gpt4(set_b_question, set_b_answers, set_a_similar_answers):
    prompt = f"""
    Question: {set_b_question}
    Answer choices: {', '.join(set_b_answers)}
    
    The 3 most similar questions and answers from the database are:
    1. {set_a_similar_answers[0][0]}
       Answer: {set_a_similar_answers[0][1]}
    2. {set_a_similar_answers[1][0]}
       Answer: {set_a_similar_answers[1][1]}
    3. {set_a_similar_answers[2][0]}
       Answer: {set_a_similar_answers[2][1]}
    
    Based on the information provided, which answer choice do you think is correct for the question above? in Format as below:
    Answer: C (Answer choice)
    """
    
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=2600,
        n=1,
        stop=None,
        temperature=0.49,
    )
    
    return response.choices[0].text.strip()

# Step 5: Compare GPT-4's answer to the actual correct answer
def compare_answers(set_b_question, set_b_answers, set_b_correct_answer, gpt4_answer):
    if gpt4_answer == set_b_correct_answer:
        return True
    else:
        return False

In [410]:
answer = ask_gpt4(set_b_only_questions[0], set_b_options[0], answers_gpt)

In [412]:
# Step 5: Compare GPT-4's answer to the actual correct answer
def compare_answers(real_answer, gpt4_answer):
    if gpt4_answer == real_answer:
        return True
    else:
        return False

In [413]:
set_b_answer[0]

'Answer: C (To generate capital gains)'

In [414]:
compare_answers(set_b_answer[0], answer)

Correct Answer: Answer: C (To generate capital gains)
GPT-4 Answer: Answer: C (To generate capital gains)


True

In [415]:
for i in range(len(set_b_only_questions)):
    set_b_only_questions_vectors = embeddings.embed_query(set_b_only_questions[i])
    question_t3 = index.query(vector=set_b_only_questions_vectors, top_k=3, namespace='questions', include_metadata=True, include_values=True)
    answer_ids = []
    for question in question_t3.matches: 
        answer_ids.append(index.query(vector=question.values, top_k=1, namespace='answers', include_metadata=True))
    answers_gpt = []
    for answer_id in answer_ids:
        answers_gpt.append(answer_id.matches[0].metadata['text'])
    answer = ask_gpt4(set_b_only_questions[i], set_b_options[i], answers_gpt)
    print(compare_answers(set_b_answer[i], answer))

Correct Answer: Answer: C (To generate capital gains)
GPT-4 Answer: Answer: D (To diversify the portfolio)
False
Correct Answer: Answer: D (Credit rating)
GPT-4 Answer: Answer: D (Credit rating)
True
Correct Answer: Answer: C (Higher tax liabilities)
GPT-4 Answer: Answer: D (Decreased portfolio volatility)
False
Correct Answer: Answer: D (Coupon payments)
GPT-4 Answer: Answer: B (Interest payments)
False
Correct Answer: Answer: A (Increased transaction costs)
GPT-4 Answer: Answer: C (Higher portfolio volatility)
False
Correct Answer: Answer: D (All of the above)
GPT-4 Answer: Answer: D (All of the above)
True
Correct Answer: Answer: C (Improved corporate governance)
GPT-4 Answer: Answer: C (Improved corporate governance)
True
Correct Answer: Answer: A (To represent the interests of shareholders)
GPT-4 Answer: Answer: A
False
Correct Answer: Answer: B (Higher potential returns)
GPT-4 Answer: Answer: B (Higher potential returns)
True
Correct Answer: Answer: B (Higher portfolio returns)
G